In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

class StopExecution(Exception):
    def _render_traceback_(self):
        return []



In [ ]:
import traceback
import warnings
import sys

def warn_with_traceback(message, category, filename, lineno, file=None, line=None):

    log = file if hasattr(file,'write') else sys.stderr
    traceback.print_stack(file=log)
    log.write(warnings.formatwarning(message, category, filename, lineno, line))

#warnings.showwarning = None

In [ ]:
import sys
sys.path.append("..")
import pickle as pkl
import hist
from analyzer.datasets import SampleManager, ProfileRepo
from analyzer.core import AnalysisResult
import math
import torch
import gpytorch
from torch.masked import masked_tensor, as_masked_tensor
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from analyzer.plotting.plots_1d import drawAs1DHist
from analyzer.plotting.plots_2d import drawAs2DHist, addTitles2D
from analyzer.plotting.plottables import PlotObject
from analyzer.plotting.mplstyles import loadStyles
import fitting.regression as regression
import fitting.plot_tools as fplt
import fitting.high_level as fhl
from gpytorch.kernels import ScaleKernel as SK
from gpytorch.kernels import RBFKernel as RBF
import linear_operator
import fitting.models as models
import fitting.utils as futil
torch.set_default_dtype(torch.float64)
plt.close("auto")
loadStyles()
%matplotlib inline

In [ ]:
res = AnalysisResult.fromFile("../results/histograms/2018mc.pkl")
profile_repo = ProfileRepo()                                                                                                                                                                            
profile_repo.loadFromDirectory("../profiles")              
sample_manager = SampleManager()
sample_manager.loadSamplesFromDirectory("../datasets", profile_repo)


hists = res.getMergedHistograms(sample_manager)

In [ ]:
bkg_name="QCDInclusive2018"
#bkg_name="Skim_QCDInclusive2018"
h = "ratio_m24_vs_m14"
complete_hist = hists[h]
integral_cr = complete_hist[bkg_name].sum()
m = (slice(None,None,hist.rebin(1)),slice(None,None,hist.rebin(1)))
#narrowed =  complete_hist[bkg_name][...,hist.loc(1000):hist.loc(3000),hist.loc(0.35):hist.loc(1.0)]
narrowed = complete_hist[bkg_name][m]

sig_hist = hists[h]["signal_312_1200_400"][m]

qcd_hist = narrowed  
true_sum = qcd_hist.sum()
print(true_sum)


In [ ]:
fig,ax= plt.subplots(1,2,figsize=(10,3))
_ = drawAs2DHist(ax[0], PlotObject.fromHist(qcd_hist))
_ = drawAs2DHist(ax[1], PlotObject.fromHist(sig_hist))

In [ ]:
import fitting.regression as fr
from fitting.windowing import EllipseWindow
from fitting.transformations import getNormalizationTransform

window = [(1300, 1750), (400, 700)]
rebin = 1
#mask_f = fr.ellipseMasker(torch.tensor([1400.0,0.6]), 200.0, 0.05)
mask_f = EllipseWindow([1150.0,0.33], [120.0,0.06])
#mask_f = EllipseWindow([1150.0,400.0], [120.0,100])
#mask_f = EllipseWindow([1150.0,0.9], [120.0,0.06])

#mask_f = fr.ellipseMasker(torch.tensor([2000.0,0.6]), 150.0,0.05)
#mask_f = fr.ellipseMasker(torch.ten can always look at another method.sor([1200.0,0.5]), 100.0,0.05)
#mask_f = fr.ellipseMasker(torch.tensor([1800.0,0.6]), 50.0,0.02)



#mask_f = fr.ellipseMasker(torch.tensor([1500.0,1400.0]), 200.0,150.0)
#mask_f = fr.ellipseMasker(torch.tensor([1450.0,0.59]), 150.0,0.05)
#mask_f=None

#mask_f = fr.ellipseMasker(torch.tensor([1200.0,0.6]), 200.0,0.05)
exclude_less=0.001
#mask_f = None
train_data = regression.makeRegressionData(qcd_hist[hist.rebin(rebin),hist.rebin(rebin)], mask_f, exclude_less=exclude_less)
test_data, m = regression.makeRegressionData(qcd_hist[hist.rebin(rebin),hist.rebin(rebin)], None, exclude_less=exclude_less, get_mask=True)
train_transform = getNormalizationTransform(train_data)
normalized_train_data = train_transform.transform(train_data)
normalized_test_data = train_transform.transform(test_data)

fig,ax = plt.subplots(3,2,figsize=(10,10),layout="tight")
fplt.simpleGrid(ax[0][0], train_data.E, train_data.X, train_data.Y)
mask = regression.getBlindedMask(test_data.X, test_data.Y, test_data.Y, test_data.V, mask_f)
squares = fplt.makeSquares(test_data.X[mask], test_data.E)
points = fplt.getPolyFromSquares(squares)
_=drawAs2DHist(ax[0][1], PlotObject.fromHist(sig_hist))
poly = matplotlib.patches.Polygon(points, edgecolor="red", fill=False)
ax[0][1].add_patch(poly)
signal_data = regression.makeRegressionData(sig_hist[hist.rebin(rebin),hist.rebin(rebin)])
signal_data = regression.DataValues(signal_data.X[m], signal_data.Y[m], signal_data.V[m], signal_data.E)
fplt.simpleGrid(ax[1][0], signal_data.E, signal_data.X, signal_data.Y)
fplt.simpleGrid(ax[1][1], normalized_train_data.E, normalized_train_data.X, normalized_train_data.Y)
fplt.simpleGrid(ax[2][1], test_data.E, test_data.X, test_data.V)

In [ ]:
print(normalized_train_data.Y[0])

In [ ]:
from torchvision.models.densenet import DenseNet

class DenseNetFeatureExtractor(DenseNet):
    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.avg_pool2d(out, kernel_size=self.avgpool_size).view(features.size(0), -1)
        return out

feature_extractor = DenseNetFeatureExtractor(block_config=(6, 6, 6), num_classes=2)
num_features = feature_extractor.classifier.in_features

In [ ]:
from gpytorch.variational.nearest_neighbor_variational_strategy import NNVariationalStrategy

class GPModel(ApproximateGP):
    def __init__(self, inducing_points, likelihood, k=256, training_batch_size=256):

        m, d = inducing_points.shape
        self.m = m
        self.k = k

        variational_distribution = gpytorch.variational.MeanFieldVariationalDistribution(m)

        if torch.cuda.is_available():
            inducing_points = inducing_points.cuda()

        variational_strategy = NNVariationalStrategy(self, inducing_points, variational_distribution, k=k,
                                                     training_batch_size=training_batch_size)
        super(GPModel, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ZeroMean()
        self.covar_module = gpytorch.kernels.MaternKernel(nu=2.5, ard_num_dims=d)

        self.likelihood = likelihood

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

    def __call__(self, x, prior=False, **kwargs):
        if x is not None:
            if x.dim() == 1:
                x = x.unsqueeze(-1)
        return self.variational_strategy(x=x, prior=False, **kwargs)

In [ ]:
class ApproximateGPModel(gpytorch.models.ApproximateGP):
    def __init__(self, inducing_points):
        variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = gpytorch.variational.VariationalStrategy(
            self, inducing_points, variational_distribution, learn_inducing_locations=True
        )
        super().__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

data_dim = test_data.X.size(-1)

class LargeFeatureExtractor(torch.nn.Sequential):
    def __init__(self):
        super(LargeFeatureExtractor, self).__init__()
        self.add_module('linear1', torch.nn.Linear(data_dim, 32))
        self.add_module('relu1', torch.nn.ReLU())
        self.add_module('linear2', torch.nn.Linear(32, 128))
        self.add_module('relu2', torch.nn.ReLU())
        self.add_module('linearx', torch.nn.Linear(128, 64))
        self.add_module('relux', torch.nn.ReLU())
        self.add_module('linear3', torch.nn.Linear(64, 32))
        self.add_module('relu3', torch.nn.ReLU())
        self.add_module('linear4', torch.nn.Linear(32, 16))
        self.add_module('relu4', torch.nn.ReLU())
        self.add_module('linear5', torch.nn.Linear(16, 2))

feature_extractor = LargeFeatureExtractor()

In [ ]:
a = torch.tensor([[0.9041,  0.96], [-0.3108, -2.4423], [-0.4821,  1.059]])
b = torch.tensor([[0.9,0.9]], dtype=torch.float64)
torch.squeeze(torch.cdist(a,b))

In [ ]:
from gpytorch.constraints import Positive
class MultipliedRBF(models.GeneralRBF):
    is_stationary = False

    def f(self, x1):
        C = torch.squeeze(torch.cdist(x1,self.point),dim=1)
        return C
    def __init__(self, *args,point_prior=None, point_constraint=None, **kwargs):
        super().__init__(*args,**kwargs)
        
        self.register_parameter(
            name='raw_point', parameter=torch.nn.Parameter(torch.zeros(*self.batch_shape, 1,2))
        )

        # set the parameter constraint to be positive, when nothing is specified
        if point_constraint is None:
            point_constraint = Positive()

        # register the constraint
        self.register_constraint("raw_point", point_constraint)
        # set the parameter prior, see
        # https://docs.gpytorch.ai/en/latest/module.html#gpytorch.Module.register_prior
        if point_prior is not None:
            self.register_prior(
                "point_prior",
                point_prior,
                lambda m: m.point,
                lambda m, v : m._set_point(v),
            )

    # now set up the 'actual' paramter
    @property
    def point(self):
        # when accessing the parameter, apply the constraint transform
        return self.raw_point_constraint.transform(self.raw_point)

    @point.setter
    def point(self, value):
        return self._set_point(value)

    def _set_length(self, value):
        if not torch.is_tensor(value):
            value = torch.as_tensor(value).to(self.raw_point)
        # when setting the paramater, transform the actual value to a raw one by applying the inverse transform
        self.initialize(raw_point=self.raw_point_constraint.inverse_transform(value))

        
    # this is the kernel function
    def forward(self, x1, x2, **params):
        fx1 = self.f(x1)
        fx2 = self.f(x2)
        c = super().forward(x1,x2,**params)
        outer = torch.outer(fx1,fx2)
        return c * outer

class GPRegressionModel(gpytorch.models.ExactGP):
        def __init__(self, train_x, train_y, likelihood):
            super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)
            self.mean_module = gpytorch.means.ConstantMean()
           
            self.covar_module = ( #gpytorch.kernels.InducingPointKernel(
                gpytorch.kernels.ScaleKernel(
                models.GeneralRBF(ard_num_dims=2) * 
                       MultipliedRBF(ard_num_dims=2) 

                ),
            #    inducing_points=train.X[::].clone(), likelihood=likelihood
           # )
            )
            #self.feature_extractor = LargeFeatureExtractor()

            # This module will scale the NN features so that they're nice values
            #self.scale_to_bounds = gpytorch.utils.grid.ScaleToBounds(-1., 1.)

        def forward(self, x):
            # We're first putting our data through a deep net (feature extractor)
            #projected_x = self.feature_extractor(x)
            #projected_x = self.scale_to_bounds(projected_x)  # Make the NN values "nice"
            projected_x = x
            mean_x = self.mean_module(projected_x)
            covar_x = self.covar_module(projected_x)
            return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [ ]:
class PVGPRegressionModel(gpytorch.models.ApproximateGP):
    def __init__(self, train_x, train_y, likelihood):
        # Define all the variational stuff
        variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(
            num_inducing_points=train_y.numel() ,
        )
        variational_strategy = gpytorch.variational.VariationalStrategy(
            self, train_x, variational_distribution
        )

        # Standard initializtation
        super(PVGPRegressionModel, self).__init__(
            variational_strategy,
          #  likelihood,
           # num_data=train_y.numel(),
              #  name_prefix="simple_regression_model"
        )
        self.likelihood = likelihood

        # Mean, covar
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            models.GeneralRBF( ard_num_dims=2)
        )

    def forward(self, x):
        mean = self.mean_module(x)  # Returns an n_data vec
        covar = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean, covar)

In [ ]:
torch.normal(train.Y,torch.sqrt(train.V) / 2)

In [ ]:
use_cuda = True 
from fitting.models import NNRBFKernel
import pyro
num_iter =  200
num_particles =  256
if torch.cuda.is_available() and use_cuda:   
    print("Loading train to gpu")
    train=normalized_train_data.toGpu()
    #train=regression.sendToGpu(train_data)
    print("Loaded train to gpu")
else:
    train = normalized_train_data

train.Y=torch.normal(train.Y,torch.sqrt(train.V) / 2)

shape = (128,32)
nnrbf = SK(NNRBFKernel(odim=2,layer_sizes=shape))

if use_cuda and False:
    smk = smk.cuda()
    nnsmk = nnsmk.cuda()
    smk.initialize_from_data(train.X, train.Y)
    nnsmk.initialize_from_data(nnsmk.feature_extractor(train.X),train.Y)

likelihood = gpytorch.likelihoods.FixedNoiseGaussianLikelihood(noise=train.V, learn_additional_noise=True)

# model=models.InducingPointModel(train.X,train.Y, 
#                                 likelihood, 
#                                 kernel=nnrbf, 
#                                 inducing=train.X)

rbf=SK(gpytorch.kernels.RBFKernel(ard_num_dims=2))
grbf=SK(models.GeneralRBF(ard_num_dims=2))

#model=models.VariationalAnyKernelModel(kernel=rbf,inducing_points=train.X[::])
#model = GPModel(inducing_points=train.X,likelihood=likelihood,)
#model= ApproximateGPModel(train.X)

#model=GPRegressionModel(train.X,train.Y,likelihood)
#likelihood = gpytorch.likelihoods.GaussianLikelihood()

print(model)


model,likelihood = regression.createModel(train, kernel=nnrbf)

#model = PVGPRegressionModel(train.X, train.Y, likelihood)

if torch.cuda.is_available() and use_cuda:       
    print("Loading model to gpu")
    model = model.cuda()
    likelihood = likelihood.cuda()
    print("Loaded model to gpu")


model,likelihood = regression.optimizeHyperparams(model,likelihood, 
                                                 train, 
                                                 mll=None, 
                                                 bar=False, 
                                                 iterations=800, lr=0.05)


def trainModelX(lr=0.01):
    optimizer = pyro.optim.Adam({"lr": 0.1})
    elbo = pyro.infer.Trace_ELBO(num_particles=num_particles, vectorize_particles=True, retain_graph=True)
    
    svi = pyro.infer.SVI(model.model, model.guide, optimizer, elbo)
    model.train()

    iterator = range(num_iter)
    for i in iterator:
        model.zero_grad()
        loss = svi.step(train.X, train.Y)
        if i % 20 == 0:
            print(loss)


def trainModel():
    optimizer = torch.optim.Adam([
    {'params': model.parameters()},
    ], lr=0.007)
    mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=train.Y.size(0))
    epochs_iter = range(1000)
    for i in epochs_iter:
        # Within each iteration, we will go over each minibatch of data
        optimizer.zero_grad()        
        output = model(train.X)
        loss = -mll(output, train.Y)
        loss.backward()
        if i % 20 == 0:
            print(loss)
        optimizer.step()
#trainModel()
print("DONE")
if torch.cuda.is_available() and use_cuda:   
    print("Loading model from gpu")
    model = model.cpu()
    likelihood = likelihood.cpu()
    print("Loaded model from gpu")

In [ ]:
#if mask_f:
  #  mask = regression.getBlindedMask(pred.X, pred.Y, test_data.Y, test_data.V, mask_f)
raw_pred_dist = regression.getPrediction(model, likelihood, normalized_test_data)
with  gpytorch.settings.cholesky_max_tries(30):
    pred_dist = futil.fixMVN(raw_pred_dist)


slope=train_transform.transform_y.slope
intercept = train_transform.transform_y.intercept
print(slope,intercept)

pred_dist = futil.affineTransformMVN(pred_dist,slope,  intercept)
#pred_dist = likelihood(pred_dist)
#pred_tr = train_transform.iTransform(regression.DataValues(normalized_test_data.X, pred_dist.mean, pred_dist.variance.detach(), normalized_test_data.E))
pred = regression.DataValues(test_data.X, pred_dist.mean, pred_dist.variance.detach(), test_data.E)

#print(all(pred_tr.Y == pred.Y))

if mask_f:
    mask = regression.getBlindedMask(pred.X, pred.Y, test_data.Y, test_data.V, mask_f)
    bpred_mean = pred.Y[mask]
    obs_mean = test_data.Y[mask]
    print(f"Total is {torch.sum(obs_mean)} -- {torch.sqrt(torch.sum(obs_mean))}")
    print(f"Approx Sigma is {torch.sum(signal_data.Y[mask])/torch.sqrt(torch.sum(obs_mean))}")
    obs_var = test_data.V[mask]

    chi2 = torch.sum((obs_mean - bpred_mean)**2 / obs_var) / torch.count_nonzero(mask)

    print(f"Global Chi^2/bins = {chi2}")

    chi2 = torch.sum((obs_mean - bpred_mean)**2 / obs_var) / torch.count_nonzero(mask)
    avg_pull = torch.sum(torch.abs((obs_mean - bpred_mean)) / torch.sqrt(obs_var)) / torch.count_nonzero(mask)
    print(f"Chi^2/bins = {chi2}")
    print(f"Avg Abs pull = {avg_pull}")
else:
    mask=None
if avg_pull < 5 or True:    
    p = fhl.makeDiagnosticPlots(pred, test_data, train_data, qcd_hist, mask)

In [ ]:
#old_pred_dist = pred_dist

In [ ]:
old_mean = old_pred_dist.mean
new_mean = pred_dist.mean
(old_mean-new_mean)**2/pred_dist.variance

In [ ]:
def makeCovarPlot(target, points, model):
    return model(target, points)
x = makeCovarPlot(
    train_transform.transform_x.transformData(torch.tensor([[1500,0.45]])), train_transform.transform_x.transformData(pred.X), model.covar_module)
x = torch.squeeze(x.to_dense().detach())
fig,ax=plt.subplots()
fplt.simpleGrid(ax, pred.E, pred.X, x)

In [ ]:
params = list(model.named_parameters_and_constraints())
for name,param,constraint in params:
    if constraint:
        real_param = constraint.transform(param)
    else:
        real_param = param
    print(f"{name.replace('raw_',''):{max(len(x[0]) for x in params)+4}} {real_param.detach().numpy().round(3)}" )

In [ ]:
import pyro.distributions as dist
import pyro
from pyro.infer import MCMC, NUTS, HMC
from pyro.infer import MCMC, NUTS, Predictive
import pyro.infer.reparam as pir
import pyro.poutine as poutine
from pyro.optim import Adam
from pyro.infer.autoguide import AutoDiagonalNormal, AutoMultivariateNormal, AutoDelta
from pyro.infer import SVI, Trace_ELBO

pyro.clear_param_store()
background = pred_dist.mean
scale=1e5
background_dist = pred_dist
#tm = test_transform.transform_y.iTransformData(pred_dist.mean)
#tc = test_transform.transform_y.iTransformVariances(pred_dist.covariance_matrix)
e = linear_operator.utils.cholesky.psd_safe_cholesky(pred_dist.covariance_matrix)
e = torch.linalg.cholesky(pred_dist.covariance_matrix)
assert(torch.allclose(pred_dist.covariance_matrix, e @ e.T))
cm = e @ e.T
mu = pred_dist.mean
background_dist = dist.MultivariateNormal(mu,cm )
if True:
    print(torch.allclose(cm, e @ e.T))
    vals,vecs = torch.linalg.eigh(cm)
    vals=vals.real
    vecs = vecs.real
    print(torch.isreal(vals))
    X = vecs @ torch.diag(torch.sqrt(vals))
    assert(torch.allclose(X @ X.T, cm))
    print(background_dist)
    
    eva = vals[:]
    eve = vecs[:].T
    
    evam = torch.sqrt(torch.diag(eva))
    #print(jnp.diag(eva).shape)
    #print(eve.shape)
    X= torch.matmul(eve, evam)
    print(X.shape)
    X=e
    #@pir.AutoReparam()
    assert(torch.allclose(X @ X.T, cm))

In [ ]:
pyro.clear_param_store()
use_variation_method = True
signal_dist = signal_data.Y
def pyro_model():           
    r= pyro.sample("rate",dist.Uniform(-20,20))  
    if use_variation_method:
        with pyro.plate("background_variations", X.shape[1]):  
            b = pyro.sample("raw_variations", dist.Normal(0,1))
        background = mu + X @ b 
    else:
        background = pyro.sample("background_dist", background_dist)
    # print("==============================================")
    # print(background)
    # print(background_dist.sample())
    # print(mu)
    # print("==============================================")    
    obs_hist = (r * signal_dist) + background
    with pyro.plate("bins", mu.shape[0]):    
        #return pyro.sample("observed", dist.Normal(torch.clamp(obs_hist,1), torch.sqrt(torch.clamp(obs_hist,1))))
        return pyro.sample("observed", dist.Poisson(torch.clamp(obs_hist,1)))

def fake_model():           
    obs_hist = mu
    with pyro.plate("bins", mu.shape[0]):    
        #return pyro.sample("observed", dist.Normal(torch.clamp(obs_hist,1), torch.sqrt(torch.clamp(obs_hist,1))))
        return pyro.sample("observed", dist.Poisson(torch.clamp(obs_hist,1)))

#obs = train_transform.transform_y.iTransformData(mu) +  0 * signal_dist
obs = mu +  1 * signal_dist

obs = torch.ceil(torch.clamp(obs,1))

conditioned = pyro.condition(pyro_model, {"observed": obs})
if False:
    fig,ax = plt.subplots(2)
    ax[0].hist([pyro_model()[0] for x in range(2000)])
    ax[1].hist([fake_model()[0] for x in range(2000)])


p = pyro.render_model(conditioned,
    render_params=True,
    render_distributions=True,
    render_deterministic=True)
guide = pyro.infer.autoguide.guides.AutoLowRankMultivariateNormal(conditioned)
p = Predictive(pyro_model, num_samples=1000,)()
f = Predictive(fake_model, num_samples=1000,)()
print(samples)

In [ ]:
print(torch.std(p["observed"][0]))
print(torch.std(f["observed"][0]))
print(pred_dist.variance[0])

In [ ]:
pyro.clear_param_store()

adam = pyro.optim.Adam({"lr": 0.01})
svi = SVI(conditioned, guide, adam, loss=Trace_ELBO())
num=1000
for j in range(num):    # calculate the loss and take a gradient step
    loss = svi.step()
    if j % (num // 5) == 0:
        print("[iteration %04d] loss: %.4f" % (j + 1, loss ))

In [ ]:
from pyro.infer import Predictive

def summary(samples):
    site_stats = {}
    for k, v in samples.items():
        site_stats[k] = {
            "mean": torch.mean(v, 0),
            "std": torch.std(v, 0),
            "5%": v.kthvalue(int(len(v) * 0.05), dim=0)[0],
            "95%": v.kthvalue(int(len(v) * 0.95), dim=0)[0],
        }
    return site_stats
predictive = Predictive(pyro_model, guide=guide,num_samples=800,)
samples = predictive()
pred_summary = summary(samples)

print(pred_summary['rate']['mean'],pred_summary['rate']['std'] )
#print(train_transform.transform_y.iTransformData(mu))
#print(train_transform.transform_y.iTransformData(mu + X @ pred_summary["raw_variations"]["mean"]))

In [ ]:
from pyro.infer import MCMC, NUTS

def hook(k,s,stage,i):
    return 
    print(s)

nuts_kernel = NUTS(conditioned,adapt_step_size=True)

mcmc = MCMC(nuts_kernel, num_samples=800, warmup_steps=400, hook_fn=hook)
mcmc.run()

hmc_samples = {k: v.detach().cpu().numpy() for k, v in mcmc.get_samples().items()}

In [ ]:
mcmc.summary()


In [ ]:
import math
r = mcmc.get_samples()["rate"]
l = r.shape[0]
p = torch.sum(r > 3.05) / l
torch.distributions.Normal(0,1).icdf(1-p)

In [ ]:
import arviz as az
data = az.from_pyro(mcmc)

one_rate = data.posterior["rate"]
ax = az.plot_trace(one_rate)
az.plot_trace(zero_rate)

In [ ]:
fix,ax=plt.subplots()
ax.hist(one_rate.to_numpy().flatten(),alpha=0.5)
ax.hist(zero_rate.to_numpy().flatten(),alpha=0.5)

In [ ]:
def is_outlier(points, thresh=3.5):
    if len(points.shape) == 1:
        points = points[:,None]
    median = torch.median(points, axis=0).values
    print(median)
    diff = torch.sum((points - median)**2, axis=-1)
    diff = torch.sqrt(diff)
    med_abs_deviation = torch.median(diff)
    print(med_abs_deviation)
    modified_z_score = 0.6745 * diff / med_abs_deviation
    return modified_z_score > thresh
def noOutliers(a, thresh = 3.5):
    return a[~is_outlier(a,thresh)]
    
s = mcmc.get_samples()
fig,ax = plt.subplots()

ax.hist(noOutliers(s['rate']))

In [ ]:
t = poutine.trace(pyro_model)
t.

In [ ]:
pyro.clear_param_store()

guide = pyro.infer.autoguide.AutoNormal(conditioned)
print(guide)
#def guide():
#    pass
num_steps = 4000
initial_lr = 0.1
gamma = 0.01  # final learning rate will be gamma * initial_lr
lrd = gamma ** (1 / num_steps)
adam = pyro.optim.ClippedAdam({'lr': initial_lr, 'lrd': lrd})
#adam = pyro.optim.Adam({"lr": 0.01})  # Consider decreasing learning rate.
elbo = pyro.infer.Trace_ELBO()
svi = pyro.infer.SVI(conditioned, guide, adam, elbo)
losses = []
for step in range(num_steps):  # Consider running for more steps.
    loss = svi.step()
    losses.append(loss)
    if step % ( num_steps // 10) == 0:
        #print(f"r = {pyro.param('r').item()}")
        print("Elbo loss: {:0.3f}".format(loss))



In [ ]:
from pyro.infer import Predictive
predictive = Predictive(conditioned, guide=guide, num_samples=1000)
p = predictive()

In [ ]:
r = p['rate'].detach().flatten()
fig,ax = plt.subplots()
ax.hist(r,bins=20)


In [ ]:
import fitting.plot_tools as fpt
import analyzer.plotting as plotting
from pathlib import Path
import importlib
import fitting

importlib.reload(fpt)
dim = 1

pred_mean,_ = fitting.regression.pointsToGrid(test_data.X, pred.Y, test_data.E)
pred_var,_ = fitting.regression.pointsToGrid(test_data.X, pred.V, test_data.E)
obs_vals,_ = fitting.regression.pointsToGrid(test_data.X, test_data.Y, test_data.E)
obs_vars, filled = fitting.regression.pointsToGrid(test_data.X, test_data.V, test_data.E)

#(figpath /"slices" / f"along_{dim}").mkdir(parents=True, exist_ok=True)
for val, f, ax in fpt.createSlices(
    pred_mean.hist,
    pred_var.hist,
    obs_vals.hist,
    obs_vars.hist,
    test_data.E,
    filled,
    mask_function=mask_f,
    observed_title="CRData", slice_dim=dim, just_window=True):
    plotting.addTitles1D(ax, plotting.PlotObject.fromHist(qcd_hist[:,sum]))
    #f.savefig(figpath /"slices" / f"along_{dim}" /  (f"slice_{round(float(val),3)}".replace(".","p") + ".pdf"))
    #plt.close(f)
    

In [ ]:
from gpytorch.models.deep_gps import DeepGPLayer, DeepGP
from gpytorch.mlls import DeepApproximateMLL
from gpytorch.means import ConstantMean, LinearMean
from gpytorch.kernels import RBFKernel, ScaleKernel
from gpytorch.variational import VariationalStrategy, CholeskyVariationalDistribution
from gpytorch.distributions import MultivariateNormal
from gpytorch.models import ApproximateGP, GP
from gpytorch.mlls import VariationalELBO, AddedLossTerm
from gpytorch.likelihoods import GaussianLikelihood
from torch.utils.data import TensorDataset, DataLoader

if torch.cuda.is_available() and use_cuda:   
    print("Loading train to gpu")
    train=normalized_train_data.toGpu()
    test=normalized_test_data.toGpu()
    #train = regression.DataValues(normalized_train_data.X.cuda(), normalized_train_data.Y.cuda(), normalized_train_data.V.cuda(), None)
    print("Loaded train to gpu")
else:
    train = normalized_train_data
    test = normalized_train_data

smoke_test = True
train_x, train_y, train_v = train.X, train.Y, train.V
test_x, test_y, test_v = test.X, test.Y, test.V


train_dataset = TensorDataset(train_x, train_y, train_v)
train_loader = DataLoader(train_dataset, batch_size=100000, shuffle=True)


class ToyDeepGPHiddenLayer(DeepGPLayer):
    def __init__(self, input_dims, output_dims, num_inducing=128, mean_type='constant'):
        if output_dims is None:
            inducing_points = torch.randn(num_inducing, input_dims) + 1
            batch_shape = torch.Size([])
        else:
            inducing_points = torch.randn(output_dims, num_inducing, input_dims) + 1
            batch_shape = torch.Size([output_dims])

        variational_distribution = CholeskyVariationalDistribution(
            num_inducing_points=num_inducing,
            batch_shape=batch_shape
        )

        variational_strategy = VariationalStrategy(
            self,
            inducing_points,
            variational_distribution,
            learn_inducing_locations=True
        )

        super(ToyDeepGPHiddenLayer, self).__init__(variational_strategy, input_dims, output_dims)

        if mean_type == 'constant':
            self.mean_module = ConstantMean(batch_shape=batch_shape)
        else:
            self.mean_module = LinearMean(input_dims)
        self.covar_module = ScaleKernel(
            RBFKernel(batch_shape=batch_shape, ard_num_dims=input_dims),
            batch_shape=batch_shape, ard_num_dims=None
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)

    def __call__(self, x, *other_inputs, **kwargs):
        """
        Overriding __call__ isn't strictly necessary, but it lets us add concatenation based skip connections
        easily. For example, hidden_layer2(hidden_layer1_outputs, inputs) will pass the concatenation of the first
        hidden layer's outputs and the input data to hidden_layer2.
        """
        if len(other_inputs):
            if isinstance(x, gpytorch.distributions.MultitaskMultivariateNormal):
                x = x.rsample()

            processed_inputs = [
                inp.unsqueeze(0).expand(gpytorch.settings.num_likelihood_samples.value(), *inp.shape)
                for inp in other_inputs
            ]

            x = torch.cat([x] + processed_inputs, dim=-1)
        return super().__call__(x, are_samples=bool(len(other_inputs)))

num_hidden_dims =  10

class DeepGP(DeepGP):
    def __init__(self, train_x_shape, train_vars):
        hidden_layer = ToyDeepGPHiddenLayer(
            input_dims=train_x_shape[-1],
            output_dims=num_hidden_dims,
            mean_type='linear',
        )
        
        middle_hidden = ToyDeepGPHiddenLayer(
            input_dims=num_hidden_dims,
            output_dims=num_hidden_dims,
            mean_type='linear',
        )

        last_layer = ToyDeepGPHiddenLayer(
            input_dims=middle_hidden.output_dims,
            output_dims=None,
            mean_type='constant',
        )
        
        super().__init__()
        
        self.hidden_layer = hidden_layer
        self.last_layer = last_layer
        self.middle_hidden = middle_hidden
        self.likelihood = gpytorch.likelihoods.FixedNoiseGaussianLikelihood(noise=train_vars)
        #self.likelihood = gpytorch.likelihoods.GaussianLi(noise=train_vars)

    def forward(self, inputs):
        hidden_rep1 = self.hidden_layer(inputs)
        x = self.middle_hidden(hidden_rep1)
        output = self.last_layer(hidden_rep1)
        return output
    
    def predict(self, test_loader):
        with torch.no_grad():
            mus = []
            variances = []
            lls = []
            for x_batch, y_batch in test_loader:
                preds = self.likelihood(self(x_batch))
                mus.append(preds.mean)
                variances.append(preds.variance)
                lls.append(model.likelihood.log_marginal(y_batch, model(x_batch)))
        
        return torch.cat(mus, dim=-1).mean(0), torch.cat(variances, dim=-1).mean(0), torch.cat(lls, dim=-1)

model = DeepGP(train_x.shape,train_v)
if torch.cuda.is_available():
    model = model.cuda()

# this is for running the notebook in our testing framework
num_epochs =  5000
num_samples = 4

optimizer = torch.optim.Adam([{'params': model.parameters()},], lr=0.05)
mll = DeepApproximateMLL(
    gpytorch.mlls.ExactMarginalLogLikelihood(model.likelihood, model))

epochs_iter = range(num_epochs)
for i in epochs_iter:
    # Within each iteration, we will go over each minibatch of data    minibatch_iter = train_loader
    
    #for x_batch, y_batch, noise in train_loader:
    with gpytorch.settings.num_likelihood_samples(num_samples):
        optimizer.zero_grad()
        output = model(train_x)
        loss = -mll(output, train_y, noise=train_v)
        loss.backward()
        optimizer.step()
    if i % (num_epochs // 10) == 0 :
        print(f"Epoch {i:3}: Loss {loss:0.3f}")
print("DONE")


In [ ]:
import gpytorch
import math


test_dataset = TensorDataset(test_x, test_y)
test_loader = DataLoader(test_dataset, batch_size=1000000)

model.eval()
#predictive_means, predictive_variances, test_lls = model.predict(test_loader)
with torch.no_grad(): 
    p = model.likelihood(model(test_x),noise=test_v)
    predictive_means = p.mean.mean(0)
    predictive_variances = p.variance.mean(0)

if torch.cuda.is_available() and use_cuda:   
    predictive_means=predictive_means.cpu()
    predictive_variances=predictive_variances.cpu()
    #train = regression.DataValues(normalized_train_data.X.cuda(), normalized_train_data.Y.cuda(), normalized_train_data.V.cuda(), None)

pred = test_transform.iTransform(
    regression.DataValues(normalized_test_data.X, predictive_means, predictive_variances, normalized_test_data.E))

import fitting.high_level as fhl
predictive_means

p = fhl.makeDiagnosticPlots(pred, test_data, train_data, qcd_hist)

In [ ]:
import pyro

class PyroGPModel(gpytorch.models.PyroGP):
    def __init__(self, train_x, train_y, likelihood, inducing_points, kernel=None, mean=None):
        variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(
            num_inducing_points=inducing_points.size(0),
        )
        variational_strategy = gpytorch.variational.VariationalStrategy(
            self, inducing_points, 
            variational_distribution, 
            learn_inducing_locations=True
        )
        super().__init__(
            variational_strategy,
            likelihood,
            num_data=train_y.numel(),
            name_prefix="simple_regression_model"
        )
        self.likelihood = likelihood
        self.mean_module = mean or gpytorch.means.ConstantMean()
        if kernel is None:
            kernel = gpytorch.kernels.ScaleKernel(
                gpytorch.kernels.RBFKernel(ard_num_dims=2)
            )
        self.covar_module = kernel

    def forward(self, x):
        mean = self.mean_module(x)  # Returns an n_data vec
        covar = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean, covar)


class StandardApproximateGP(gpytorch.models.ApproximateGP):
    def __init__(self, inducing_points,kernel=None, mean=None):
        variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = gpytorch.variational.VariationalStrategy(
            self, 
            inducing_points, 
            variational_distribution, 
            learn_inducing_locations=True
        )
        super().__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
        
        self.mean_module = mean or gpytorch.means.ConstantMean()
        if kernel is None:
            kernel = gpytorch.kernels.ScaleKernel(
                gpytorch.kernels.RBFKernel(ard_num_dims=2)
            )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
        
use_cuda = True 

if torch.cuda.is_available() and use_cuda:   
    print("Loading train to gpu")
    train=regression.sendToGpu(normalized_train_data)
    print("Loaded train to gpu")
else:
    train = normalized_train_data

NNRBF = fitting.models.wrapNN("NNRBFKernel", gpytorch.kernels.RBFKernel)
nnrbf = SK(NNRBF(odim=2,layer_sizes=(256,128,64,32)))
nnrbf = SK(NNRBF(odim=2,layer_sizes=(32,16)))

likelihood = gpytorch.likelihoods.FixedNoiseGaussianLikelihood(noise=train.V)
model=StandardApproximateGP(
                 #train.X,
                 #train.Y, 
                 #likelihood, 
                 train.X[::4],
                 kernel=grbf
              )

if torch.cuda.is_available() and use_cuda:   
    print("Loading model to gpu")
    model = model.cuda()
    likelihood = likelihood.cuda()
    print("Loaded model to gpu")

def trainModel(model, likelihood, lr=0.01, num_iter = 1):
    optimizer = pyro.optim.Adam({"lr": lr})
    elbo = pyro.infer.Trace_ELBO(num_particles=128, vectorize_particles=True, retain_graph=True)
    svi = pyro.infer.SVI(model.model, model.guide, optimizer, elbo)
    model.train()
    likelihood.train()

    iterator = range(num_iter)
    for i in iterator:
        model.zero_grad()
        loss = svi.step(train.X, train.Y)
        if i % (num_iter // 10) == 0 :
            print(f"Iter {i}: Loss {loss:2}")

    return model

def trainModel2(model, likelihood,lr=0.1,num_iter = 1):
    model.train()
    likelihood.train()
    optimizer = torch.optim.Adam([
        {'params': model.parameters()},
        {'params': likelihood.parameters()},
    ], lr=lr)
    
    # Our loss object. We're using the VariationalELBO
    mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=train.Y.size(0))
    
    
    iterator = range(num_iter)
    for i in iterator:
        optimizer.zero_grad()
        output = model(train.X)
        loss = -mll(output, train.Y)
        loss.backward()
        optimizer.step()
        
        if i % (num_iter // 10) == 0 :
            print(f"Iter {i}: Loss {loss:2}")
    
    print(f"Iter {i}: Loss {loss:2}")
    return model
    

print(model)
model = trainModel2(model,likelihood,lr=0.025,num_iter=10000)



print("DONE")
#with linear_operator.settings.max_cg_iterations(2000):
#    model,likelihood = regression.optimizeHyperparams(model,likelihood, train, bar=False, iterations=250, lr=0.05)
print("DONE")
if torch.cuda.is_available() and use_cuda:   
    print("Loading model to gpu")
    model = model.cpu()
    likelihood = likelihood.cpu()
    print("Loaded model to gpu")

In [ ]:
params = list(model.named_parameters_and_constraints())
for name,param,constraint in params:
    if constraint:
        real_param = constraint.transform(param)
    else:
        real_param = param
    print(f"{name.replace('raw_',''):{max(len(x[0]) for x in params)+4}} {real_param.detach().numpy().round(3)}" )

In [ ]:

pred_dist = regression.getPrediction(model, likelihood, normalized_test_data)
pred = test_transform.iTransform(    regression.DataValues(normalized_test_data.X, pred_dist.mean, pred_dist.variance, normalized_test_data.E))

if window:
    mask = regression.getBlindedMask(pred.X, pred.Y, test_data.Y, test_data.V, mask_f)
    bpred_mean = pred.Y[mask]
    obs_mean = test_data.Y[mask]
    obs_var = test_data.V[mask]
    chi2 = torch.sum((obs_mean - bpred_mean)**2 / obs_var) / torch.count_nonzero(mask)
    avg_pull = torch.sum(torch.abs((obs_mean - bpred_mean)) / torch.sqrt(obs_var)) / torch.count_nonzero(mask)
    print(f"Chi^2/bins = {chi2}")
    print(f"Avg Abs pull = {avg_pull}")
if avg_pull < 5:    
    p = fhl.makeDiagnosticPlots(pred, test_data, train_data, qcd_hist, mask)

In [ ]:

p = fhl.makeDiagnosticPlots(pred, test_data, train_data, qcd_hist, mask)
_,ax1 = p["gpr_mean"]
indp=model.covar_module.inducing_points
ind = test_transform.transform_x.iTransformData(indp).detach()

#ax1.scatter(ind[:,0], ind[:,1])
